In [13]:
#Include
using Plots, LightGraphs, SparseArrays, SimpleWeightedGraphs
using Statistics, BenchmarkTools, LinearAlgebra, ProgressMeter
using Distributions, Base.Threads, CSV, StatsBase
using Base.GC, JLD2, FileIO, TSne
plotly();

In [14]:
function tm()
    println("Number of threads = $(nthreads())")

    # sin1(x::Float64) = ccall((:sin, Base.Math.libm), Float64, (Float64,), x)
    # cos1(x::Float64) = ccall((:cos, Base.Math.libm), Float64, (Float64,), x)
    sin1(x::Float64) = ccall(:sin, Float64, (Float64,), x)
    cos1(x::Float64) = ccall(:cos, Float64, (Float64,), x)

    function test1!(y, x)
        # @assert length(y) == length(x)
        for i = 1:length(x)
            y[i] = sin1(x[i])^2 + cos1(x[i])^2
        end
        y
    end

    function testn!(y::Vector{Float64}, x::Vector{Float64})
        # @assert length(y) == length(x)
        Threads.@threads for i = 1:length(x)
            y[i] = sin1(x[i])^2 + cos1(x[i])^2
        end
        y
    end
    n = 10^7
    x = rand(n)
    y = zeros(n)
    @time test1!(y, x)
    @time testn!(y, x)
    @time test1!(y, x)
    @time testn!(y, x);
    flush(stdout)
end

tm()

Number of threads = 8
  0.371176 seconds
  0.143884 seconds (539 allocations: 35.440 KiB)
  0.546685 seconds
  0.135704 seconds (1 allocation: 48 bytes)


In [84]:
function read_tree(filename,c)
    lines = readlines(open(filename))
    count = Dict()
    for i = 1:length(lines)
        l = split(lines[i],c)
        count[l[1]] = 1 
        count[l[2]] = 1
    end
    
    n = length(count)
    G = SimpleGraph(n)
    for i = 1:length(lines)
        l = parse.(Int64,split(lines[i],c))
        add_edge!(G, l[1]+1,l[2]+1)
    end
    
    return G
end

struct MultipleDijkstraState{T <: Real,U <: Integer}
    dists::Array{T,2}
    parents::Array{U,2}
end

function parallel_dp_shortest_paths(g,distmx::AbstractMatrix{T}) where T <: Real
    n_v = nv(g)
    N = n_v
    p = Progress(N);
    update!(p,0)
    jj = Threads.Atomic{Int}(0)

    l = Threads.SpinLock()

    # TODO: remove `Int` once julialang/#23029 / #23032 are resolved
    dists   = Array{T,2}(undef,(Int(n_v), Int(n_v)))
    parents = Array{Int64,2}(undef,(Int(n_v), Int(n_v)))

    state = LightGraphs.dijkstra_shortest_paths(g,[1],distmx)
    dists[1, :] = state.dists 
    parents[1, :] = state.parents
        
    Threads.@threads for i in 2:n_v
        state = LightGraphs.dijkstra_shortest_paths(g,[i],distmx)
        dists[i, :] = state.dists
        parents[i, :] = state.parents
        Threads.atomic_add!(jj, 1)
        Threads.threadid() == 1 && update!(p, jj[])
    end

    #result = MultipleDijkstraState(dists, parents)
    return MultipleDijkstraState(dists, parents)
end

parallel_dp_shortest_paths (generic function with 1 method)

In [101]:
function gid(D,w,x,y)
    return 0.5*(D[w,x]+D[w,y]-D[x,y])
end

function metric_to_structure_C(d,tol)
    n,_ = size(d)
    W = zeros(2*n,2*n)
    W[1:n,1:n] = d
    G = SimpleGraph(n)
    
    p = randperm(n)
    
    x = p[1]
    y = p[2]
    z = p[3]
    V = collect(4:n)
    
    for i = 4:n
        V[i-3] = p[i]
    end
    
    nextRoots = collect(2*n:-1:n+1)
    
    C = []
    
    G,W,nextRoots,C = recursive_step_C(G,V,x,y,z,W,1,nextRoots,C,tol)
    
    return C
end

function recursive_step_C(G,V,x,y,z,W,ztype,nextRoots,C,tol = 0.000000001)
    n = Int(size(W,1)/2)
    r = pop!(nextRoots)
    add_vertex!(G)
    add_edge!(G,x,r)
    add_edge!(G,y,r)
    add_edge!(G,z,r)
    
    if ztype == 2
        rem_edge!(G,x,y)
    end

    X1 = []
    X2 = []
    Y1 = []
    Y2 = []
    Z1 = []
    Z2 = []
    R1 = []
    
    W[r,x] = gid(W,x,y,z)
    W[x,r] = W[r,x]
    
    replaced_root = false
    
    if abs(W[r,x]) < tol && !replaced_root
        replaced_root = true
        W[r,x] = 0
        W[x,r] = 0
        rem_edge!(G,x,r)
        rem_edge!(G,y,r)
        rem_edge!(G,z,r)
        rem_vertex!(G,r)
        add_edge!(G,x,y)
        add_edge!(G,z,x)
        
        push!(nextRoots,r)
        r = x
    end
        
    W[y,r] = gid(W,y,x,z)
    W[r,y] = W[y,r]
    
    if abs(W[r,y]) < tol && !replaced_root
        replaced_root = true
        W[r,x] = 0
        W[x,r] = 0
        W[r,y] = 0
        W[y,r] = 0
        rem_edge!(G,x,r)
        rem_edge!(G,y,r)
        rem_edge!(G,z,r)
        rem_vertex!(G,r)
        add_edge!(G,x,y)
        add_edge!(G,z,y)
        
        push!(nextRoots,r)
        r = y
    end
    
    W[r,z] = gid(W,z,x,y)
    W[z,r] = W[r,z]
    
    if abs(W[r,z]) < tol && !replaced_root
        replaced_root = true
        W[r,x] = 0
        W[x,r] = 0
        W[r,y] = 0
        W[y,r] = 0
        W[r,z] = 0
        W[z,r] = 0
        rem_edge!(G,x,r)
        rem_edge!(G,y,r)
        rem_edge!(G,z,r)
        rem_vertex!(G,r)
        add_edge!(G,z,y)
        add_edge!(G,z,x)
        
        push!(nextRoots,r)
        r = z
    end
    
    xc = false
    yc = false
    zc = false
    
    for w in V
        a = gid(W,w,x,y)
        b = gid(W,w,y,z)
        c = gid(W,w,z,x)
        
        if abs(a-b) < tol && abs(b-c) < tol && abs(c-a) < tol
            if a < tol && b < tol && c < tol && !replaced_root
                replaced_root = true
                W[w,n+1:2*n] = W[r,n+1:2*n] 
                W[n+1:2*n,w] = W[n+1:2*n,r]
                W[:,r] = zeros(2*n)
                W[r,:] = zeros(2*n)
                rem_edge!(G,x,r)
                rem_edge!(G,y,r)
                rem_edge!(G,z,r)
                rem_vertex!(G,r)
                push!(nextRoots,r)
                r = w
                add_edge!(G,x,r)
                add_edge!(G,y,r)
                add_edge!(G,z,r)
            else
                push!(R1,w)
                W[w,r] = (a+b+c)/3
                W[r,w] = W[w,r]
            end
        elseif a == maximum([a,b,c])
            if abs(b-c) > tol
                zc = true
                push!(C,(w,x,y,z))
            end
            if abs(W[w,z] - b) < tol || abs(W[w,z] - c) < tol
                push!(Z1,w)
            else
                push!(Z2,w)
            end
            W[w,r] = a
            W[r,w] = a
        elseif b == maximum([a,b,c])
            if abs(a-c) > tol
                push!(C,(w,x,y,z))
                xc = true
            end
            if abs(W[w,z] - a) < tol || abs(W[w,z] - c) < tol
                push!(X1,w)
            else
                push!(X2,w)
            end
            W[w,r] = b
            W[r,w] = b
        elseif c == maximum([a,b,c])
            if abs(b-a) > tol
                push!(C,(w,x,y,z))
                yc = false
            end
            if abs(W[w,z] - b) < tol || abs(W[w,z] - a) < tol
                push!(Y1,w)
            else
                push!(Y2,w)
            end
            W[w,r] = c
            W[r,w] = c
        end
    end
        
    G,W,nextRoots,C = zone1_recurse_C(G,W,R1,r,nextRoots,C,tol)
    if !xc
        G,W,nextRoots,C = zone1_recurse_C(G,W,X1,x,nextRoots,C,tol)
        G,W,nextRoots,C = zone2_recurse_C(G,W,X2,x,r,nextRoots,C,tol)
    end
    if !yc
        G,W,nextRoots,C = zone1_recurse_C(G,W,Y1,y,nextRoots,C,tol)
        G,W,nextRoots,C = zone2_recurse_C(G,W,Y2,y,r,nextRoots,C,tol)
    end
    if !zc
        G,W,nextRoots,C = zone1_recurse_C(G,W,Z1,z,nextRoots,C,tol)
        G,W,nextRoots,C = zone2_recurse_C(G,W,Z2,z,r,nextRoots,C,tol)
    end
    
    return G,W,nextRoots,C
end

function zone1_recurse_C(G,W,V,x,nextRoots,C,tol = 0.000000001)
    ztype = 1 
    if length(V) == 0
        return G,W,nextRoots,C
    end
    
    if length(V) == 1
        add_edge!(G,x,V[1])
        return G,W,nextRoots,C
    end
    
    y = V[1]
    z = V[2]
    
    n = Int(size(W,1)/2)
    r = pop!(nextRoots)
    add_vertex!(G)
    add_edge!(G,x,r)
    add_edge!(G,y,r)
    add_edge!(G,z,r)
    
    if ztype == 2
        rem_edge!(G,x,y)
    end

    X1 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    X2 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    Y1 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    Y2 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    Z1 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    Z2 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    R1 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    C1 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    
    W[r,x] = gid(W,x,y,z)
    W[x,r] = W[r,x]
    
    replaced_root = false
    
    if abs(W[r,x]) < tol && !replaced_root
        replaced_root = true
        W[r,x] = 0
        W[x,r] = 0
        rem_edge!(G,x,r)
        rem_edge!(G,y,r)
        rem_edge!(G,z,r)
        rem_vertex!(G,r)
        add_edge!(G,x,y)
        add_edge!(G,z,x)
        
        push!(nextRoots,r)
        r = x
    end
        
    W[y,r] = gid(W,y,x,z)
    W[r,y] = W[y,r]
    
    if abs(W[r,y]) < tol && !replaced_root
        replaced_root = true
        W[r,x] = 0
        W[x,r] = 0
        W[r,y] = 0
        W[y,r] = 0
        rem_edge!(G,x,r)
        rem_edge!(G,y,r)
        rem_edge!(G,z,r)
        rem_vertex!(G,r)
        add_edge!(G,x,y)
        add_edge!(G,z,y)
        
        push!(nextRoots,r)
        r = y
    end
    
    W[r,z] = gid(W,z,x,y)
    W[z,r] = W[r,z]
    
    if abs(W[r,z]) < tol && !replaced_root
        replaced_root = true
        W[r,x] = 0
        W[x,r] = 0
        W[r,y] = 0
        W[y,r] = 0
        W[r,z] = 0
        W[z,r] = 0
        rem_edge!(G,x,r)
        rem_edge!(G,y,r)
        rem_edge!(G,z,r)
        rem_vertex!(G,r)
        add_edge!(G,z,y)
        add_edge!(G,z,x)
        
        push!(nextRoots,r)
        r = z
    end
    
    xc = false
    yc = false
    zc = false
    
    Ṽ = V[3:end]
    @inbounds Threads.@threads for w in Ṽ 
        a = gid(W,w,x,y)
        b = gid(W,w,y,z)
        c = gid(W,w,z,x)
        
        if abs(a-b) < tol && abs(b-c) < tol && abs(c-a) < tol
            if a < tol && b < tol && c < tol && !replaced_root
                replaced_root = true
                W[w,n+1:2*n] = W[r,n+1:2*n] 
                W[n+1:2*n,w] = W[n+1:2*n,r]
                W[:,r] = zeros(2*n)
                W[r,:] = zeros(2*n)
                rem_edge!(G,x,r)
                rem_edge!(G,y,r)
                rem_edge!(G,z,r)
                rem_vertex!(G,r)
                push!(nextRoots,r)
                r = w
                add_edge!(G,x,r)
                add_edge!(G,y,r)
                add_edge!(G,z,r)
            else
                push!(R1[Threads.threadid()],w)
                W[w,r] = (a+b+c)/3
                W[r,w] = W[w,r]
            end
        elseif a == maximum([a,b,c])
            if abs(b-c) > tol
                zc = true
                if x > n
                    push!(C1[Threads.threadid()],(w,1,y,z))
                    push!(C1[Threads.threadid()],(w,2,y,z))
                    push!(C1[Threads.threadid()],(w,3,y,z))
                    for u in Ṽ
                        push!(C1[Threads.threadid()],(w,u,y,z))
                    end
                else
                    push!(C1[Threads.threadid()],(w,x,y,z))
                end
            end
            if abs(W[w,z] - b) < tol && abs(W[w,z] - c) < tol
                push!(Z1[Threads.threadid()],w)
            else
                push!(Z2[Threads.threadid()],w)
            end
            W[w,r] = a
            W[r,w] = a
        elseif b == maximum([a,b,c])
            if abs(a-c) > tol
                xc = true
                if x > n
                    push!(C1[Threads.threadid()],(w,1,y,z))
                    push!(C1[Threads.threadid()],(w,2,y,z))
                    push!(C1[Threads.threadid()],(w,3,y,z))
                    for u in Ṽ
                        push!(C1[Threads.threadid()],(w,u,y,z))
                    end
                else
                    push!(C1[Threads.threadid()],(w,x,y,z))
                end
            end
            if abs(W[w,z] - a) < tol && abs(W[w,z] - c) < tol
                push!(X1[Threads.threadid()],w)
            else
                push!(X2[Threads.threadid()],w)
            end
            W[w,r] = b
            W[r,w] = b
        elseif c == maximum([a,b,c])
            if abs(b-a) > tol
                yc = true
                if x > n
                    push!(C1[Threads.threadid()],(w,1,y,z))
                    push!(C1[Threads.threadid()],(w,2,y,z))
                    push!(C1[Threads.threadid()],(w,3,y,z))
                    for u in Ṽ
                        push!(C1[Threads.threadid()],(w,u,y,z))
                    end
                else
                    push!(C1[Threads.threadid()],(w,x,y,z))
                end
            end
            if abs(W[w,z] - b) < tol && abs(W[w,z] - a) < tol
                push!(Y1[Threads.threadid()],w)
            else
                push!(Y2[Threads.threadid()],w)
            end
            W[w,r] = c
            W[r,w] = c
        end
    end
    
    R1p = R1[1]
    X1p = X1[1]
    Y1p = Y1[1]
    Z1p = Z1[1]
    X2p = X2[1]
    Y2p = Y2[1]
    Z2p = Z2[1]
    C = append!(C,C1[1])
    for i = 2:16
        R1p = append!(R1p,R1[i])
        X1p = append!(X1p,X1[i])
        Y1p = append!(Y1p,Y1[i])
        Z1p = append!(Z1p,Z1[i])
        X2p = append!(X2p,X2[i])
        Y2p = append!(Y2p,Y2[i])
        Z2p = append!(Z2p,Z2[i])
        C = append!(C,C1[i])
    end
        
    G,W,nextRoots,C = zone1_recurse_C(G,W,R1p,r,nextRoots,C,tol)
    if !xc
        G,W,nextRoots,C = zone1_recurse_C(G,W,X1p,x,nextRoots,C,tol)
        G,W,nextRoots,C = zone2_recurse_C(G,W,X2p,x,r,nextRoots,C,tol)
    end
    if !yc
        G,W,nextRoots,C = zone1_recurse_C(G,W,Y1p,y,nextRoots,C,tol)
        G,W,nextRoots,C = zone2_recurse_C(G,W,Y2p,y,r,nextRoots,C,tol)
    end
    if !zc
        G,W,nextRoots,C = zone1_recurse_C(G,W,Z1p,z,nextRoots,C,tol)
        G,W,nextRoots,C = zone2_recurse_C(G,W,Z2p,z,r,nextRoots,C,tol)
    end
    
    return G,W,nextRoots,C
end

function zone2_recurse_C(G,W,V,x,y,nextRoots,C,tol = 0.000000001)
    ztype = 2
    if length(V) == 0
        return G,W,nextRoots,C
    end
    
    p = sortperm(W[y,V])

    z = V[p[1]]
    
    n = Int(size(W,1)/2)
    r = pop!(nextRoots)
    add_vertex!(G)
    add_edge!(G,x,r)
    add_edge!(G,y,r)
    add_edge!(G,z,r)
    
    if ztype == 2
        rem_edge!(G,x,y)
    end

    X1 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    X2 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    Y1 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    Y2 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    Z1 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    Z2 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    R1 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    C1 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    
    W[r,x] = gid(W,x,y,z)
    W[x,r] = W[r,x]
    
    replaced_root = false
    
    if abs(W[r,x]) < tol && !replaced_root
        replaced_root = true
        W[r,x] = 0
        W[x,r] = 0
        rem_edge!(G,x,r)
        rem_edge!(G,y,r)
        rem_edge!(G,z,r)
        rem_vertex!(G,r)
        add_edge!(G,x,y)
        add_edge!(G,z,x)
        
        push!(nextRoots,r)
        r = x
    end
        
    W[y,r] = gid(W,y,x,z)
    W[r,y] = W[y,r]
    
    if abs(W[r,y]) < tol && !replaced_root
        replaced_root = true
        W[r,x] = 0
        W[x,r] = 0
        W[r,y] = 0
        W[y,r] = 0
        rem_edge!(G,x,r)
        rem_edge!(G,y,r)
        rem_edge!(G,z,r)
        rem_vertex!(G,r)
        add_edge!(G,x,y)
        add_edge!(G,z,y)
        
        push!(nextRoots,r)
        r = y
    end
    
    W[r,z] = gid(W,z,x,y)
    W[z,r] = W[r,z]
    
    if abs(W[r,z]) < tol && !replaced_root
        replaced_root = true
        W[r,x] = 0
        W[x,r] = 0
        W[r,y] = 0
        W[y,r] = 0
        W[r,z] = 0
        W[z,r] = 0
        rem_edge!(G,x,r)
        rem_edge!(G,y,r)
        rem_edge!(G,z,r)
        rem_vertex!(G,r)
        add_edge!(G,z,y)
        add_edge!(G,z,x)
        
        push!(nextRoots,r)
        r = z
    end
    
    xc = false
    yc = false
    zc = false
    
    Ṽ = V[2:end]
    for i = 1:length(Ṽ)
        Ṽ[i] = V[p[i+1]]
    end
    @inbounds Threads.@threads for w in Ṽ 
        a = gid(W,w,x,y)
        b = gid(W,w,y,z)
        c = gid(W,w,z,x)
        
        if abs(a-b) < tol && abs(b-c) < tol && abs(c-a) < tol
            if a < tol && b < tol && c < tol && !replaced_root
                replaced_root = true
                W[w,n+1:2*n] = W[r,n+1:2*n] 
                W[n+1:2*n,w] = W[n+1:2*n,r]
                W[:,r] = zeros(2*n)
                W[r,:] = zeros(2*n)
                rem_edge!(G,x,r)
                rem_edge!(G,y,r)
                rem_edge!(G,z,r)
                rem_vertex!(G,r)
                push!(nextRoots,r)
                r = w
                add_edge!(G,x,r)
                add_edge!(G,y,r)
                add_edge!(G,z,r)
            else
                push!(R1[Threads.threadid()],w)
                W[w,r] = (a+b+c)/3
                W[r,w] = W[w,r]
            end
        elseif a == maximum([a,b,c])
            if abs(b-c) > tol
                zc = true
                if x > n && y > n
                    push!(C1[Threads.threadid()],(w,2,1,z))
                    push!(C1[Threads.threadid()],(w,3,1,z))
                    push!(C1[Threads.threadid()],(w,1,2,z))
                    push!(C1[Threads.threadid()],(w,3,2,z))
                    push!(C1[Threads.threadid()],(w,1,3,z))
                    push!(C1[Threads.threadid()],(w,2,3,z))
                elseif y > n
                    push!(C1[Threads.threadid()],(w,x,1,z))
                    push!(C1[Threads.threadid()],(w,x,2,z))
                    push!(C1[Threads.threadid()],(w,x,3,z))
                    for u in Ṽ
                        push!(C1[Threads.threadid()],(w,x,u,z))
                    end
                elseif x > n
                    push!(C1[Threads.threadid()],(w,1,y,z))
                    push!(C1[Threads.threadid()],(w,2,y,z))
                    push!(C1[Threads.threadid()],(w,3,y,z))
                    for u in Ṽ
                        push!(C1[Threads.threadid()],(w,u,y,z))
                    end
                else
                    push!(C1[Threads.threadid()],(w,x,y,z))
                end
            end
            if abs(W[w,z] - b) < tol || abs(W[w,z] - c) < tol
                push!(Z1[Threads.threadid()],w)
            else
                push!(Z2[Threads.threadid()],w)
            end
            W[w,r] = a
            W[r,w] = a
        elseif b == maximum([a,b,c])
            if abs(c-a) > tol
                xc = true
                if x > n && y > n
                    push!(C1[Threads.threadid()],(w,2,1,z))
                    push!(C1[Threads.threadid()],(w,3,1,z))
                    push!(C1[Threads.threadid()],(w,1,2,z))
                    push!(C1[Threads.threadid()],(w,3,2,z))
                    push!(C1[Threads.threadid()],(w,1,3,z))
                    push!(C1[Threads.threadid()],(w,2,3,z))
                elseif y > n
                    push!(C1[Threads.threadid()],(w,x,1,z))
                    push!(C1[Threads.threadid()],(w,x,2,z))
                    push!(C1[Threads.threadid()],(w,x,3,z))
                    for u in Ṽ
                        push!(C1[Threads.threadid()],(w,x,u,z))
                    end
                elseif x > n
                    push!(C1[Threads.threadid()],(w,1,y,z))
                    push!(C1[Threads.threadid()],(w,2,y,z))
                    push!(C1[Threads.threadid()],(w,3,y,z))
                    for u in Ṽ
                        push!(C1[Threads.threadid()],(w,u,y,z))
                    end
                else
                    push!(C1[Threads.threadid()],(w,x,y,z))
                end
            end
            if abs(W[w,z] - a) < tol || abs(W[w,z] - c) < tol
                push!(X1[Threads.threadid()],w)
            else
                push!(X2[Threads.threadid()],w)
            end
            W[w,r] = b
            W[r,w] = b
        elseif c == maximum([a,b,c])
            if abs(b-a) > tol
                yc = true
                if x > n && y > n
                    push!(C1[Threads.threadid()],(w,2,1,z))
                    push!(C1[Threads.threadid()],(w,3,1,z))
                    push!(C1[Threads.threadid()],(w,1,2,z))
                    push!(C1[Threads.threadid()],(w,3,2,z))
                    push!(C1[Threads.threadid()],(w,1,3,z))
                    push!(C1[Threads.threadid()],(w,2,3,z))
                elseif y > n
                    push!(C1[Threads.threadid()],(w,x,1,z))
                    push!(C1[Threads.threadid()],(w,x,2,z))
                    push!(C1[Threads.threadid()],(w,x,3,z))
                    for u in Ṽ
                        push!(C1[Threads.threadid()],(w,x,u,z))
                    end
                elseif x > n
                    push!(C1[Threads.threadid()],(w,1,y,z))
                    push!(C1[Threads.threadid()],(w,2,y,z))
                    push!(C1[Threads.threadid()],(w,3,y,z))
                    for u in Ṽ
                        push!(C1[Threads.threadid()],(w,u,y,z))
                    end
                else
                    push!(C1[Threads.threadid()],(w,x,y,z))
                end
            end
            if abs(W[w,z] - b) < tol || abs(W[w,z] - a) < tol
                push!(Y1[Threads.threadid()],w)
            else
                push!(Y2[Threads.threadid()],w)
            end
            W[w,r] = c
            W[r,w] = c
        end
    end
    
    R1p = R1[1]
    X1p = X1[1]
    Y1p = Y1[1]
    Z1p = Z1[1]
    X2p = X2[1]
    Y2p = Y2[1]
    Z2p = Z2[1]
    C = append!(C,C1[1])
    for i = 2:16
        R1p = append!(R1p,R1[i])
        X1p = append!(X1p,X1[i])
        Y1p = append!(Y1p,Y1[i])
        Z1p = append!(Z1p,Z1[i])
        X2p = append!(X2p,X2[i])
        Y2p = append!(Y2p,Y2[i])
        Z2p = append!(Z2p,Z2[i])
        C = append!(C,C1[i])
    end
        
    G,W,nextRoots,C = zone1_recurse_C(G,W,R1p,r,nextRoots,C,tol)
    if !xc
        G,W,nextRoots,C = zone1_recurse_C(G,W,X1p,x,nextRoots,C,tol)
        G,W,nextRoots,C = zone2_recurse_C(G,W,X2p,x,r,nextRoots,C,tol)
    end
    if !yc
        G,W,nextRoots,C = zone1_recurse_C(G,W,Y1p,y,nextRoots,C,tol)
        G,W,nextRoots,C = zone2_recurse_C(G,W,Y2p,y,r,nextRoots,C,tol)
    end
    if !zc
        G,W,nextRoots,C = zone1_recurse_C(G,W,Z1p,z,nextRoots,C,tol)
        G,W,nextRoots,C = zone2_recurse_C(G,W,Z2p,z,r,nextRoots,C,tol)
    end

    return G,W,nextRoots,C
end

function tree_find_rand(D,w=1)
    n = size(D,2)
    Consts = []
    for i = 1:n^2
        i = rand(1:n)
        j = rand(1:n)
        k = rand(1:n)
        w = rand(1:n)
        
        if i != j && j !=k && k != i
            push!(Consts,(w,i,j,k))
        end
    end
    
    return Consts
end

tree_find_rand (generic function with 2 methods)

In [104]:
function tree_proj(D,Z2,tol)
    error = 0
    maxE = 0
    n,n = size(D)
    Consts = keys(Z2)
    for (w,i,j,k) in Consts
        a = 2*gid(D,w,i,j)
        b = 2*gid(D,w,j,k)
        c = 2*gid(D,w,i,k)
        if a == maximum([a,b,c])
            d = b-c
            error += abs(d)
            b = b - d/2
            c = c + d/2
        elseif b == maximum([a,b,c])
            d = a-c
            error += abs(d)
            a = a - d/2
            c = c + d/2
        else
            d = a-b
            error += abs(d)
            a = a - d/2
            b = b + d/2
        end
        
        if abs(d) < tol
            delete!(Z2,(w,i,j,k))
        end
        
        if abs(d) > maxE
            maxE = abs(d)
        end
        
        ad = 2*gid(D,w,i,j) - a
        bd = 2*gid(D,w,j,k) - b
        cd = 2*gid(D,w,i,k) - c
                
        D[w,i] -= ad/3
        D[w,j] -= ad/3
        D[i,w] -= ad/3
        D[j,w] -= ad/3
        D[i,j] += ad/3
        D[j,i] += ad/3
                    
        D[w,k] -= bd/3
        D[w,j] -= bd/3
        D[k,w] -= bd/3
        D[j,w] -= bd/3
        D[k,j] += bd/3
        D[j,k] += bd/3

        D[w,i] -= cd/3
        D[w,k] -= cd/3
        D[i,w] -= cd/3
        D[k,w] -= cd/3
        D[i,k] += cd/3
        D[k,i] += cd/3
    end
    
    return D,maxE,error,Z2
end

function tree_proj_full(D,w=rand(1:n))
    error = 0
    maxE = 0
    n,n = size(D)
    Δ = zeros(n,n)
    for i = 1:n
        for j=1:i-1
            for k=1:j-1
                a = gid(D,w,i,j)
                b = gid(D,w,j,k)
                c = gid(D,w,i,k)
                d = 0
                if a == maximum([a,b,c])
                    d = b-c
                    error += abs(d)
                    b = b - d/2
                    c = c + d/2
                elseif b == maximum([a,b,c])
                    d = a-c
                    error += abs(d)
                    a = a - d/2
                    c = c + d/2
                else
                    d = a-b
                    error += abs(d)
                    a = a - d/2
                    b = b + d/2
                end
                
                if abs(d) > maxE
                    maxE = abs(d)
                end
                
                ad = gid(D,w,i,j) - a
                bd = gid(D,w,j,k) - b
                cd = gid(D,w,i,k) - c
                
                D[w,i] -= ad/3
                D[w,j] -= ad/3
                D[i,w] -= ad/3
                D[j,w] -= ad/3
                D[i,j] += ad/3
                D[j,i] += ad/3
                
                D[w,k] -= bd/3
                D[w,j] -= bd/3
                D[k,w] -= bd/3
                D[j,w] -= bd/3
                D[k,j] += bd/3
                D[j,k] += bd/3
                
                D[w,i] -= cd/3
                D[w,k] -= cd/3
                D[i,w] -= cd/3
                D[k,w] -= cd/3
                D[i,k] += cd/3
                D[k,i] += cd/3
            end
        end
    end
    
    @show(error,maxE)
    
    return D,maxE
end

function enumerate_paths(s)
    P = Array{Any,1}(undef,size(s.parents, 1))
    Threads.@threads for v = 1:size(s.parents, 1)
        P[v] = LightGraphs.enumerate_paths(s, v)
    end
    
    return P
end

function h1(G,Z′)
    (n,n) = size(G)
    @inbounds Threads.@threads for i = 1:n
        for j = 1:i-1
            c = min(G[j,i],Z′[j,i])
            G[j,i] -= c
            G[i,j] -= c
            Z′[j,i] -= c            
        end
    end
    return G
end

function h2(F,E,Λ,Π)
    @inbounds Threads.@threads for i = 1:n
        for j = 1:i-1
            δ1 = (F[j,i]+E[j,i])/2
            δ2 = (F[j,i]-E[j,i])/2
            c1 = min(δ1,Λ[j,i])
            c2 = min(δ2,Π[j,i])
            E[j,i] = E[j,i] - c1 + c2
            E[i,j] = E[i,j] - c1 + c2
            F[j,i] = F[j,i] - c1 - c2
            F[i,j] = F[i,j] - c1 - c2
            Λ[j,i] -= c1
            Π[j,i] -= c2
        end
    end
end

function tree_proj_for_rand(D,Consts)
    error = 0
    maxE = 0
    n,n = size(D)
    for (w,i,j,k) in Consts
        a = gid(D,w,i,j)
        b = gid(D,w,j,k)
        c = gid(D,w,i,k)
        if a == maximum([a,b,c])
            d = b-c
            error += abs(d)
            b = b - d/2
            c = c + d/2
        elseif b == maximum([a,b,c])
            d = a-c
            error += abs(d)
            a = a - d/2
            c = c + d/2
        else
            d = a-b
            error += abs(d)
            a = a - d/2
            b = b + d/2
        end
        
        if abs(d) > maxE
            maxE = abs(d)
        end
        
        ad = gid(D,w,i,j) - a
        bd = gid(D,w,j,k) - b
        cd = gid(D,w,i,k) - c
                
        D[w,i] -= ad/3
        D[w,j] -= ad/3
        D[i,w] -= ad/3
        D[j,w] -= ad/3
        D[i,j] += ad/3
        D[j,i] += ad/3
                    
        D[w,k] -= bd/3
        D[w,j] -= bd/3
        D[k,w] -= bd/3
        D[j,w] -= bd/3
        D[k,j] += bd/3
        D[j,k] += bd/3

        D[w,i] -= cd/3
        D[w,k] -= cd/3
        D[i,w] -= cd/3
        D[k,w] -= cd/3
        D[i,k] += cd/3
        D[k,i] += cd/3
    end
    
    return D,maxE
end

tree_proj_for_rand (generic function with 1 method)

In [19]:
#returns (w,x,y,z) so that gid(D,w,x,y) == gid(D,w,x,z)
function get_constraints(D,w=rand(1:n))
    n,n = size(D)
    C = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    @inbounds Threads.@threads for i = 1:n
        for j = 1:i-1
            for k = 1:j-1
                a = gid(D,w,i,j)
                b = gid(D,w,j,k)
                c = gid(D,w,k,i)
                
                if a == b
                    push!(C[Threads.threadid()],(w,j,i,k))
                end
                
                if c == b
                    push!(C[Threads.threadid()],(w,k,i,j))
                end
                
                if a == c
                    push!(C[Threads.threadid()],(w,i,j,k))
                end
            end
        end
    end
    
    Cp = C[1]
    for i = 2:16
        C = append!(Cp,C[i])
    end
    
    return Cp
end

function tree_proj_have_structure(D,Consts)
    error = 0
    maxE = 0
    n,n = size(D)
    for (w,i,j,k) in Consts
        a = 2*gid(D,w,i,j)
        b = 4*gid(D,w,j,k)
        c = 2*gid(D,w,i,k)
        d = a-c
        error += abs(d)
        a = a - d/2
        c = c + d/2
        
        if abs(d) > maxE
            maxE = abs(d)
        end
        
        d2 = b - a - c
        if d2 < 0
        
        ad = 2*gid(D,w,i,j) - a
        cd = 2*gid(D,w,i,k) - c
                
        D[w,i] -= ad/3
        D[w,j] -= ad/3
        D[i,w] -= ad/3
        D[j,w] -= ad/3
        D[i,j] += ad/3
        D[j,i] += ad/3

        D[w,i] -= cd/3
        D[w,k] -= cd/3
        D[i,w] -= cd/3
        D[k,w] -= cd/3
        D[i,k] += cd/3
        D[k,i] += cd/3
        
    end
    
    return D,maxE
end
end

tree_proj_have_structure (generic function with 1 method)

In [105]:
function BregmanCC(D,W⁻¹,C,γ,tol,filename)
    (n,n) = size(D)
    g = LightGraphs.SimpleGraphs.CompleteGraph(n)
    G = copy(D)
    Z = Dict()
    Z2 = Dict()
    Z′ = zeros(n,n)
    Λ = zeros(n,n)
    Π = zeros(n,n)
    F = -1*γ*(W⁻¹.*C)
    nthds::Int = Threads.nthreads()

    maxD = tol+1
    maxE = tol+1
    count = 1
    z = 1

    # First time through constraints
    #parallel_cyclic_triple_loopE!(G,current_corrections,next_corrections)
    
    while((maxD > tol || maxE > tol) && z > 0 && count < 2000)
        maxD,G,Z,maxE,Z2 = loop(g,G,Z,Z2,Z′,Λ,Π,F,D,W⁻¹,C,γ,count,filename,tol)
        z = length(Z2)
        @show(avg_distortion(G,D))
        #maxD,G,Z,maxE = loop(g,G,Z,Z′,Λ,Π,F,D,W⁻¹,C,γ,count,filename,tol)
        flush(stdout)
        count+=1
    end
    return (g,count,Z,F,G)
end

#Some random tree constraints and then ocassionally all of them
function loop(g,G,Z,Z2,Z′,Λ,Π,F,D,W⁻¹,C,γ,count,filename,tol)
    for k = 1:10
        #Project onto the cycles we have
        for p in keys(Z)
            z = Z[p]
            l = length(p)
            u = p[1]
            v = p[l]
            d = -1*G[u,v]
            m = W⁻¹[u,v]
            for i = 1:l-1
                d = d + G[p[i], p[i+1]]
                m += W⁻¹[p[i], p[i+1]]
            end
            c = min(d/m,z)
            for i = 1:l-1
                ch = W⁻¹[p[i],p[i+1]]*c
                G[p[i],p[i+1]] -= ch
                G[p[i+1],p[i]] -= ch
            end
            ch = W⁻¹[u,v]*c
            G[u,v] += ch
            G[v,u] += ch
            if z == c || z - c < 1e-14
                delete!(Z,p)
            else
                Z[p] -= c
            end
        end
    end
    
    h1(G,Z′)

    #Find new broken cycles
    FS =LightGraphs.Parallel.floyd_warshall_shortest_paths(g,G) 
    U = FS.dists
    P = enumerate_paths(FS)
    
    maxD2 = norm(U - G,1)
    maxD = maximum(G-U)
    
    #Project onto new broken cycles
    for i = 1:n
        for j = 1:i-1
            if G[j,i] - U[j,i] > 0  
                p = P[j][i] 
                l = length(p)
                u = p[1]
                v = p[l]
                d = -1*G[u,v]
                m = W⁻¹[u,v]
                for k = 1:l-1
                    d = d + G[p[k], p[k+1]]
                    m += W⁻¹[p[k], p[k+1]]
                end
                if d < 0
                    c=d/m
                    for k = 1:l-1
                        G[p[k],p[k+1]] -= W⁻¹[p[k],p[k+1]]*c
                        G[p[k+1],p[k]] -= W⁻¹[p[k],p[k+1]]*c
                    end
                    G[p[1],p[l]] += W⁻¹[p[1],p[l]]*c
                    G[p[l],p[1]] += W⁻¹[p[1],p[l]]*c
                    if haskey(Z,p)
                        Z[p] = Z[p] - c
                    else
                        Z[p] = -1*c
                    end
                end
            end   
        end
    end
    
    P = 0
    U = 0

    flush(stdout)
        
    #Make sure that F = |E|
    #E = G - D
    #h2(F,E,Λ,Π)
    #G = E + D  

    Consts = metric_to_structure_C(G,100*tol)
    for k in Consts
        if !haskey(Z2,k)
            Z2[k] = 0
        end
    end
    @show(length(Consts))
    
    maxE = 0
    terror = 0
    for i = 1:10
        G2,maxE,terror,Z2 = tree_proj(G,Z2,tol)
        G = G2
    end
    
    for i = 1:10
        Consts = tree_find_rand(G)
        G2,_ = tree_proj_for_rand(G,Consts)
        G = G2
    end

    
    @show((length(Z2),maxE,terror))
    flush(stdout)
    
    Consts = 0
    
    maxE2 = 0
    if count %100 == 0
        @time G2,maxE2 = tree_proj_full(G)
        @show(maxE2)
        G = G2
    end
    
    @show((maxD,maxD2,length(Z),count))
    open(filename,"a") do f
        lenz = length(Z)
        if count %100 == 0
            write(f, "maxD = $maxD, maxD2 = $maxD2, length(z) = $lenz, count = $count, maxE = $maxE, maxE2 = $maxE2 \n")
        else
            write(f, "maxD = $maxD, maxD2 = $maxD2, length(z) = $lenz, count = $count, maxE = $maxE \n")
        end
    end

    return maxD,G,Z,maxE,Z2
end

function loop_have_structure(g,G,Z,Z′,Λ,Π,F,D,W⁻¹,C,γ,count,filename)
    for k = 1:10
        #Project onto the cycles we have
        for p in keys(Z)
            z = Z[p]
            l = length(p)
            u = p[1]
            v = p[l]
            d = -1*G[u,v]
            m = W⁻¹[u,v]
            for i = 1:l-1
                d = d + G[p[i], p[i+1]]
                m += W⁻¹[p[i], p[i+1]]
            end
            c = min(d/m,z)
            for i = 1:l-1
                ch = W⁻¹[p[i],p[i+1]]*c
                G[p[i],p[i+1]] -= ch
                G[p[i+1],p[i]] -= ch
            end
            ch = W⁻¹[u,v]*c
            G[u,v] += ch
            G[v,u] += ch
            if z == c || z - c < 1e-14
                delete!(Z,p)
            else
                Z[p] -= c
            end
        end
    end
    
    h1(G,Z′)

    #Find new broken cycles
    FS = LightGraphs.Parallel.floyd_warshall_shortest_paths(g,G) 
    U = FS.dists
    P = enumerate_paths(FS)
    
    maxD2 = norm(U - G,1)
    maxD = maximum(G-U)
    
    #Project onto new broken cycles
    for i = 1:n
        for j = 1:i-1
            if G[j,i] - U[j,i] > 0  
                p = P[j][i] 
                l = length(p)
                u = p[1]
                v = p[l]
                d = -1*G[u,v]
                m = W⁻¹[u,v]
                for k = 1:l-1
                    d = d + G[p[k], p[k+1]]
                    m += W⁻¹[p[k], p[k+1]]
                end
                if d < 0
                    c=d/m
                    for k = 1:l-1
                        G[p[k],p[k+1]] -= W⁻¹[p[k],p[k+1]]*c
                        G[p[k+1],p[k]] -= W⁻¹[p[k],p[k+1]]*c
                    end
                    G[p[1],p[l]] += W⁻¹[p[1],p[l]]*c
                    G[p[l],p[1]] += W⁻¹[p[1],p[l]]*c
                    if haskey(Z,p)
                        Z[p] = Z[p] - c
                    else
                        Z[p] = -1*c
                    end
                end
            end   
        end
    end
    
    P = 0
    U = 0

    flush(stdout)
        
    #Make sure that F = |E|
    E = G - D
    h2(F,E,Λ,Π)
    G = E + D  

    @time Consts = get_constraints(D)
    @show(length(Consts))
    @time for i = 1:10
        G2,maxE = tree_proj_have_structure(G,Consts)
        G = G2
    end
    
    flush(stdout)
    
    Consts = 0
    
    @show((maxD,maxD2,length(Z),count,maxE))
    open(filename,"a") do f
        lenz = length(Z)
        write(f, "maxD = $maxD, maxD2 = $maxD2, length(z) = $lenz, count = $count, maxE = $maxE \n")
    end
    
    return maxD,G,Z,maxE
end

loop_have_structure (generic function with 1 method)

In [97]:
G = read_tree("./../hyperbolics-master/data/edges/bio-diseasome.edges"," ")
n = nv(G)
E = ne(G)
@show((n,E));

(n, E) = (516, 1188)


In [98]:
F = parallel_dp_shortest_paths(G,adjacency_matrix(G))
A = F.dists;

Progress:  47%|███████████████████▏                     |  ETA: 0:00:00

In [99]:
filename = "bio-diseasome.txt"
open(filename,"w") do f
    write(f,"")
end

0

In [106]:
W⁻¹ = ones(size(A))
A = convert(Array{Float64,2},A)

@time R = BregmanCC(A,W⁻¹,W⁻¹,1,1e-5,filename)

length(Consts) = 12341
(length(Z2), maxE, terror) = (292, 0.2699113627027785, 18.111799511228526)
(maxD, maxD2, length(Z), count) = (0.0, 0.0, 0, 1)
avg_distortion(G, D) = 0.06674708997785439
length(Consts) = 62970
(length(Z2), maxE, terror) = (62153, 1.7202983242544594, 2423.135605220842)
(maxD, maxD2, length(Z), count) = (2.972537918785684, 16281.21803420479, 6636, 2)
avg_distortion(G, D) = 0.07615312652373409
length(Consts) = 855
(length(Z2), maxE, terror) = (62649, 1.32544771851554, 1120.9446170445092)
(maxD, maxD2, length(Z), count) = (2.29079901455669, 1527.5263935040148, 5527, 3)
avg_distortion(G, D) = 0.07926105900585745
length(Consts) = 135724
(length(Z2), maxE, terror) = (194997, 0.8028219346964223, 1566.8185635522154)
(maxD, maxD2, length(Z), count) = (1.540106089085751, 362.15964931216035, 3993, 4)
avg_distortion(G, D) = 0.08093405549310767
length(Consts) = 516
(length(Z2), maxE, terror) = (191062, 0.8438799150609011, 836.7814094264731)
(maxD, maxD2, length(Z), count) = (0.

(maxD, maxD2, length(Z), count) = (0.00686400411322019, 1.06247009071977, 356, 37)
avg_distortion(G, D) = 0.08269124643914212
length(Consts) = 2346
(length(Z2), maxE, terror) = (4554, 0.04072966808353273, 0.7820832573797445)
(maxD, maxD2, length(Z), count) = (0.010553179216028497, 1.012097205993973, 350, 38)
avg_distortion(G, D) = 0.08269077923912989
length(Consts) = 993
(length(Z2), maxE, terror) = (3369, 0.037350020682312746, 0.6635560604689714)
(maxD, maxD2, length(Z), count) = (0.004753936320153862, 0.9588209388709494, 334, 39)
avg_distortion(G, D) = 0.08269085846765259
length(Consts) = 3523
(length(Z2), maxE, terror) = (3607, 0.034061513755412776, 0.5814132678490025)
(maxD, maxD2, length(Z), count) = (0.004451635262693543, 1.1493736315217797, 353, 40)
avg_distortion(G, D) = 0.0826906351348553
length(Consts) = 635
(length(Z2), maxE, terror) = (2637, 0.031214439224292345, 0.4874025129605759)
(maxD, maxD2, length(Z), count) = (0.006469291767139396, 0.972561526225096, 333, 41)
avg_dis

(maxD, maxD2, length(Z), count) = (6.015120560398657e-5, 0.9046284674335288, 237, 73)
avg_distortion(G, D) = 0.08268579738657311
length(Consts) = 0
(length(Z2), maxE, terror) = (146, 0.0005373874784879717, 0.006405817406434977)
(maxD, maxD2, length(Z), count) = (9.268410493890045e-5, 0.9044553532945996, 236, 74)
avg_distortion(G, D) = 0.08268574454854705
length(Consts) = 0
(length(Z2), maxE, terror) = (136, 0.00045665828658747465, 0.005461973882479931)
(maxD, maxD2, length(Z), count) = (5.623896657747096e-5, 0.9050734061483541, 251, 75)
avg_distortion(G, D) = 0.08268570153672544
length(Consts) = 0
(length(Z2), maxE, terror) = (130, 0.0003885987504834709, 0.004798609703570378)
(maxD, maxD2, length(Z), count) = (2.9422157455716302e-5, 0.9043186936709569, 245, 76)
avg_distortion(G, D) = 0.0826856512267295
length(Consts) = 6
(length(Z2), maxE, terror) = (120, 0.00034703306344474605, 0.00419194255638744)
(maxD, maxD2, length(Z), count) = (5.04040664761618e-5, 0.905785579870036, 278, 77)
avg

length(Consts) = 0
(length(Z2), maxE, terror) = (8, 3.7516521956604265e-5, 0.00020189602659614891)
(maxD, maxD2, length(Z), count) = (0.000413125781141499, 0.9077131425206268, 310, 109)
avg_distortion(G, D) = 0.08268448955227416
length(Consts) = 0
(length(Z2), maxE, terror) = (8, 3.915826542533196e-5, 0.0002033783665043032)
(maxD, maxD2, length(Z), count) = (0.0004005725584863562, 0.9069362886577517, 332, 110)
avg_distortion(G, D) = 0.0826844677788688
length(Consts) = 0
(length(Z2), maxE, terror) = (8, 3.568468584758122e-5, 0.0001912954973581904)
(maxD, maxD2, length(Z), count) = (0.0002470939513194992, 0.9055342723641797, 308, 111)
avg_distortion(G, D) = 0.08268445091166962
length(Consts) = 0
(length(Z2), maxE, terror) = (8, 3.5909100495779e-5, 0.00018408327332775087)
(maxD, maxD2, length(Z), count) = (0.00037971065028985507, 0.9073390512082316, 333, 112)
avg_distortion(G, D) = 0.08268442888314055
length(Consts) = 0
(length(Z2), maxE, terror) = (8, 3.6012536805074546e-5, 0.00018711949

({516, 132870} undirected simple Int64 graph, 142, Dict{Any,Any}([486, 498, 499]=>0.00186453,[165, 11, 345]=>1.09481e-7,[63, 197, 196]=>1.9489e-8,[5, 328, 327]=>2.02176e-7,[327, 328, 511]=>2.09678e-5,[198, 197, 484]=>1.20462e-7,[5, 257, 401]=>1.13289e-6,[196, 197, 393]=>4.6235e-8,[61, 197, 196]=>0.00395821,[34, 257, 401]=>0.00432779…), [-1.0 -1.0 … -1.0 -1.0; -1.0 -1.0 … -1.0 -1.0; … ; -1.0 -1.0 … -1.0 -1.0; -1.0 -1.0 … -1.0 -1.0], [0.0 1.02506 … 9.79633 11.2121; 1.02506 -0.0501276 … 8.79643 10.2122; … ; 9.79633 8.79643 … 0.0 7.14072; 11.2121 10.2122 … 7.14072 0.0])

In [107]:
Dtree = R[end]
for i = 1:n
    Dtree[i,i] = 0
end

In [108]:
Dtree

516×516 Array{Float64,2}:
  0.0       1.02506  11.0071   11.0071   …  10.1778   9.79633  11.2121 
  1.02506   0.0      10.0072   10.0072       9.17786  8.79643  10.2122 
 11.0071   10.0072    0.0       1.00237     10.1985   9.81703  11.2328 
 11.0071   10.0072    1.00237   0.0         10.1985   9.81703  11.2328 
 11.0071   10.0072    0.99943   1.00236     10.1985   9.81702  11.2328 
 11.4438   10.4439   11.4645   11.4645   …   5.50554  7.37247   7.32064
 10.4438    9.44394  10.4645   10.4645       4.50554  6.37247   6.32065
 10.4618    9.4619   10.4825   10.4825       2.98073  6.39043   6.3386 
 10.4657    9.46579  10.4864   10.4864       2.98463  6.39432   6.34249
 10.2568    9.2569   10.2775   10.2775       2.77573  6.18542   6.1336 
  9.8996    8.8997    9.9203    9.92029  …   2.41853  5.82822   5.7764 
 10.4657    9.46576  10.4864   10.4864       2.98459  6.39429   6.34246
  6.00129   5.00139   7.00703   7.00702      6.17767  5.79624   7.212  
  ⋮                                   

In [53]:
function gid(D,w,x,y)
    return 0.5*(D[w,x]+D[w,y]-D[x,y])
end

function metric_to_structure(d)
    n,_ = size(d)
    global W = zeros(2*n,2*n)
    W[1:n,1:n] = d
    G = SimpleGraph(n)
    
    p = randperm(n)
    
    x = p[1]
    y = p[2]
    z = p[3]
    V = collect(4:n)
    for i = 4:n
        V[i-3] = p[i]
    end
    
    global nextRoots = collect(2*n:-1:n+1)
    
    G= recursive_step(G,V,x,y,z,1)
    
    @show((nv(G),ne(G)))

    for i = 1:nv(G)
        if has_edge(G,i,i)
            rem_edge!(G,i,i)
        end
    end
    
    return G,W
end

function recursive_step(G,V,x,y,z,ztype,tol = 0.000000001)
    n = Int(size(W,1)/2)
    r = pop!(nextRoots)
    add_vertex!(G)
    add_edge!(G,x,r)
    add_edge!(G,y,r)
    add_edge!(G,z,r)
    
    if ztype == 2
        rem_edge!(G,x,y)
    end

    X1 = []
    X2 = []
    Y1 = []
    Y2 = []
    Z1 = []
    Z2 = []
    R1 = []
    
    W[r,x] = gid(W,x,y,z)
    W[x,r] = W[r,x]
    
    replaced_root = false
    
    if abs(W[r,x]) < tol && !replaced_root
        replaced_root = true
        W[r,x] = 0
        W[x,r] = 0
        rem_edge!(G,x,r)
        rem_edge!(G,y,r)
        rem_edge!(G,z,r)
        rem_vertex!(G,r)
        add_edge!(G,x,y)
        add_edge!(G,z,x)
        
        push!(nextRoots,r)
        r = x
    end
        
    W[y,r] = gid(W,y,x,z)
    W[r,y] = W[y,r]
    
    if abs(W[r,y]) < tol && !replaced_root
        replaced_root = true
        W[r,x] = 0
        W[x,r] = 0
        W[r,y] = 0
        W[y,r] = 0
        rem_edge!(G,x,r)
        rem_edge!(G,y,r)
        rem_edge!(G,z,r)
        rem_vertex!(G,r)
        add_edge!(G,x,y)
        add_edge!(G,z,y)
        
        push!(nextRoots,r)
        r = y
    end
    
    W[r,z] = gid(W,z,x,y)
    W[z,r] = W[r,z]
    
    if abs(W[r,z]) < tol && !replaced_root
        replaced_root = true
        W[r,x] = 0
        W[x,r] = 0
        W[r,y] = 0
        W[y,r] = 0
        W[r,z] = 0
        W[z,r] = 0
        rem_edge!(G,x,r)
        rem_edge!(G,y,r)
        rem_edge!(G,z,r)
        rem_vertex!(G,r)
        add_edge!(G,z,y)
        add_edge!(G,z,x)
        
        push!(nextRoots,r)
        r = z
    end
    
    for w in V
        a = gid(W,w,x,y)
        b = gid(W,w,y,z)
        c = gid(W,w,z,x)
        
        if abs(a-b) < tol && abs(b-c) < tol && abs(c-a) < tol
            if a < tol && b < tol && c < tol && !replaced_root
                replaced_root = true
                W[w,n+1:2*n] = W[r,n+1:2*n] 
                W[n+1:2*n,w] = W[n+1:2*n,r]
                W[:,r] = zeros(2*n)
                W[r,:] = zeros(2*n)
                rem_edge!(G,x,r)
                rem_edge!(G,y,r)
                rem_edge!(G,z,r)
                rem_vertex!(G,r)
                push!(nextRoots,r)
                r = w
                add_edge!(G,x,r)
                add_edge!(G,y,r)
                add_edge!(G,z,r)
            else
                push!(R1,w)
                W[w,r] = (a+b+c)/3
                W[r,w] = W[w,r]
            end
        elseif a == maximum([a,b,c])
            if abs(W[w,z] - b) < tol || abs(W[w,z] - c) < tol
                push!(Z1,w)
            else
                push!(Z2,w)
            end
            W[w,r] = a
            W[r,w] = a
        elseif b == maximum([a,b,c])
            if abs(W[w,z] - a) < tol || abs(W[w,z] - c) < tol
                push!(X1,w)
            else
                push!(X2,w)
            end
            W[w,r] = b
            W[r,w] = b
        elseif c == maximum([a,b,c])
            if abs(W[w,z] - b) < tol || abs(W[w,z] - a) < tol
                push!(Y1,w)
            else
                push!(Y2,w)
            end
            W[w,r] = c
            W[r,w] = c
        end
    end
        
    G = zone1_recurse(G,R1,r)
    G = zone1_recurse(G,X1,x)
    G = zone1_recurse(G,Y1,y)
    G = zone1_recurse(G,Z1,z)
    
    G = zone2_recurse(G,X2,x,r)
    G = zone2_recurse(G,Y2,y,r)
    G = zone2_recurse(G,Z2,z,r)
    
    return G
end

function zone1_recurse(G,V,x,tol = 0.000000001)
    ztype = 1 
    if length(V) == 0
        return G
    end
    
    if length(V) == 1
        add_edge!(G,x,V[1])
        return G
    end
    
    y = V[1]
    z = V[2]
    
    n = Int(size(W,1)/2)
    r = pop!(nextRoots)
    add_vertex!(G)
    add_edge!(G,x,r)
    add_edge!(G,y,r)
    add_edge!(G,z,r)
    
    if ztype == 2
        rem_edge!(G,x,y)
    end

    X1 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    X2 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    Y1 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    Y2 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    Z1 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    Z2 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    R1 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    
    W[r,x] = gid(W,x,y,z)
    W[x,r] = W[r,x]
    
    replaced_root = false
    
    if abs(W[r,x]) < tol && !replaced_root
        replaced_root = true
        W[r,x] = 0
        W[x,r] = 0
        rem_edge!(G,x,r)
        rem_edge!(G,y,r)
        rem_edge!(G,z,r)
        rem_vertex!(G,r)
        add_edge!(G,x,y)
        add_edge!(G,z,x)
        
        push!(nextRoots,r)
        r = x
    end
        
    W[y,r] = gid(W,y,x,z)
    W[r,y] = W[y,r]
    
    if abs(W[r,y]) < tol && !replaced_root
        replaced_root = true
        W[r,x] = 0
        W[x,r] = 0
        W[r,y] = 0
        W[y,r] = 0
        rem_edge!(G,x,r)
        rem_edge!(G,y,r)
        rem_edge!(G,z,r)
        rem_vertex!(G,r)
        add_edge!(G,x,y)
        add_edge!(G,z,y)
        
        push!(nextRoots,r)
        r = y
    end
    
    W[r,z] = gid(W,z,x,y)
    W[z,r] = W[r,z]
    
    if abs(W[r,z]) < tol && !replaced_root
        replaced_root = true
        W[r,x] = 0
        W[x,r] = 0
        W[r,y] = 0
        W[y,r] = 0
        W[r,z] = 0
        W[z,r] = 0
        rem_edge!(G,x,r)
        rem_edge!(G,y,r)
        rem_edge!(G,z,r)
        rem_vertex!(G,r)
        add_edge!(G,z,y)
        add_edge!(G,z,x)
        
        push!(nextRoots,r)
        r = z
    end
    
    Ṽ = V[3:end]
    @inbounds Threads.@threads for w in Ṽ 
        a = gid(W,w,x,y)
        b = gid(W,w,y,z)
        c = gid(W,w,z,x)
        
        if abs(a-b) < tol && abs(b-c) < tol && abs(c-a) < tol
            if a < tol && b < tol && c < tol && !replaced_root
                replaced_root = true
                W[w,n+1:2*n] = W[r,n+1:2*n] 
                W[n+1:2*n,w] = W[n+1:2*n,r]
                W[:,r] = zeros(2*n)
                W[r,:] = zeros(2*n)
                rem_edge!(G,x,r)
                rem_edge!(G,y,r)
                rem_edge!(G,z,r)
                rem_vertex!(G,r)
                push!(nextRoots,r)
                r = w
                add_edge!(G,x,r)
                add_edge!(G,y,r)
                add_edge!(G,z,r)
            else
                push!(R1[Threads.threadid()],w)
                W[w,r] = (a+b+c)/3
                W[r,w] = W[w,r]
            end
        elseif a == maximum([a,b,c])
            if abs(W[w,z] - b) < tol && abs(W[w,z] - c) < tol
                push!(Z1[Threads.threadid()],w)
            else
                push!(Z2[Threads.threadid()],w)
            end
            W[w,r] = a
            W[r,w] = a
        elseif b == maximum([a,b,c])
            if abs(W[w,z] - a) < tol && abs(W[w,z] - c) < tol
                push!(X1[Threads.threadid()],w)
            else
                push!(X2[Threads.threadid()],w)
            end
            W[w,r] = b
            W[r,w] = b
        elseif c == maximum([a,b,c])
            if abs(W[w,z] - b) < tol && abs(W[w,z] - a) < tol
                push!(Y1[Threads.threadid()],w)
            else
                push!(Y2[Threads.threadid()],w)
            end
            W[w,r] = c
            W[r,w] = c
        end
    end
    
    R1p = R1[1]
    X1p = X1[1]
    Y1p = Y1[1]
    Z1p = Z1[1]
    X2p = X2[1]
    Y2p = Y2[1]
    Z2p = Z2[1]
    for i = 2:16
        R1p = append!(R1p,R1[i])
        X1p = append!(X1p,X1[i])
        Y1p = append!(Y1p,Y1[i])
        Z1p = append!(Z1p,Z1[i])
        X2p = append!(X2p,X2[i])
        Y2p = append!(Y2p,Y2[i])
        Z2p = append!(Z2p,Z2[i])
    end
        
        
    G = zone1_recurse(G,R1p,r)
    G = zone1_recurse(G,X1p,x)
    G = zone1_recurse(G,Y1p,y)
    G = zone1_recurse(G,Z1p,z)
    
    G = zone2_recurse(G,X2p,x,r)
    G = zone2_recurse(G,Y2p,y,r)
    G = zone2_recurse(G,Z2p,z,r)
    
    return G
end

function zone2_recurse(G,V,x,y,tol = 0.000000001)
    ztype = 2
    if length(V) == 0
        return G
    end
    
    p = sortperm(W[y,V])

    z = V[p[1]]
    
    n = Int(size(W,1)/2)
    r = pop!(nextRoots)
    add_vertex!(G)
    add_edge!(G,x,r)
    add_edge!(G,y,r)
    add_edge!(G,z,r)
    
    if ztype == 2
        rem_edge!(G,x,y)
    end

    X1 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    X2 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    Y1 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    Y2 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    Z1 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    Z2 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    R1 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    
    W[r,x] = gid(W,x,y,z)
    W[x,r] = W[r,x]
    
    replaced_root = false
    
    if abs(W[r,x]) < tol && !replaced_root
        replaced_root = true
        W[r,x] = 0
        W[x,r] = 0
        rem_edge!(G,x,r)
        rem_edge!(G,y,r)
        rem_edge!(G,z,r)
        rem_vertex!(G,r)
        add_edge!(G,x,y)
        add_edge!(G,z,x)
        
        push!(nextRoots,r)
        r = x
    end
        
    W[y,r] = gid(W,y,x,z)
    W[r,y] = W[y,r]
    
    if abs(W[r,y]) < tol && !replaced_root
        replaced_root = true
        W[r,x] = 0
        W[x,r] = 0
        W[r,y] = 0
        W[y,r] = 0
        rem_edge!(G,x,r)
        rem_edge!(G,y,r)
        rem_edge!(G,z,r)
        rem_vertex!(G,r)
        add_edge!(G,x,y)
        add_edge!(G,z,y)
        
        push!(nextRoots,r)
        r = y
    end
    
    W[r,z] = gid(W,z,x,y)
    W[z,r] = W[r,z]
    
    if abs(W[r,z]) < tol && !replaced_root
        replaced_root = true
        W[r,x] = 0
        W[x,r] = 0
        W[r,y] = 0
        W[y,r] = 0
        W[r,z] = 0
        W[z,r] = 0
        rem_edge!(G,x,r)
        rem_edge!(G,y,r)
        rem_edge!(G,z,r)
        rem_vertex!(G,r)
        add_edge!(G,z,y)
        add_edge!(G,z,x)
        
        push!(nextRoots,r)
        r = z
    end
    
    Ṽ = V[2:end]
    for i = 2:length(V)
        Ṽ[i-1] = V[p[i]]
    end
    @inbounds Threads.@threads for w in Ṽ 
        a = gid(W,w,x,y)
        b = gid(W,w,y,z)
        c = gid(W,w,z,x)
        
        if abs(a-b) < tol && abs(b-c) < tol && abs(c-a) < tol
            if a < tol && b < tol && c < tol && !replaced_root
                replaced_root = true
                W[w,n+1:2*n] = W[r,n+1:2*n] 
                W[n+1:2*n,w] = W[n+1:2*n,r]
                W[:,r] = zeros(2*n)
                W[r,:] = zeros(2*n)
                rem_edge!(G,x,r)
                rem_edge!(G,y,r)
                rem_edge!(G,z,r)
                rem_vertex!(G,r)
                push!(nextRoots,r)
                r = w
                add_edge!(G,x,r)
                add_edge!(G,y,r)
                add_edge!(G,z,r)
            else
                push!(R1[Threads.threadid()],w)
                W[w,r] = (a+b+c)/3
                W[r,w] = W[w,r]
            end
        elseif a == maximum([a,b,c])
            if abs(W[w,z] - b) < tol || abs(W[w,z] - c) < tol
                push!(Z1[Threads.threadid()],w)
            else
                push!(Z2[Threads.threadid()],w)
            end
            W[w,r] = a
            W[r,w] = a
        elseif b == maximum([a,b,c])
            if abs(W[w,z] - a) < tol || abs(W[w,z] - c) < tol
                push!(X1[Threads.threadid()],w)
            else
                push!(X2[Threads.threadid()],w)
            end
            W[w,r] = b
            W[r,w] = b
        elseif c == maximum([a,b,c])
            if abs(W[w,z] - b) < tol || abs(W[w,z] - a) < tol
                push!(Y1[Threads.threadid()],w)
            else
                push!(Y2[Threads.threadid()],w)
            end
            W[w,r] = c
            W[r,w] = c
        end
    end
    
    R1p = R1[1]
    X1p = X1[1]
    Y1p = Y1[1]
    Z1p = Z1[1]
    X2p = X2[1]
    Y2p = Y2[1]
    Z2p = Z2[1]
    for i = 2:16
        R1p = append!(R1p,R1[i])
        X1p = append!(X1p,X1[i])
        Y1p = append!(Y1p,Y1[i])
        Z1p = append!(Z1p,Z1[i])
        X2p = append!(X2p,X2[i])
        Y2p = append!(Y2p,Y2[i])
        Z2p = append!(Z2p,Z2[i])
    end
        
        
    G = zone1_recurse(G,R1p,r)
    G = zone1_recurse(G,X1p,x)
    G = zone1_recurse(G,Y1p,y)
    G = zone1_recurse(G,Z1p,z)
    
    G = zone2_recurse(G,X2p,x,r)
    G = zone2_recurse(G,Y2p,y,r)
    G = zone2_recurse(G,Z2p,z,r)

    return G
end

zone2_recurse (generic function with 2 methods)

In [54]:
function MAP(Dnew,G)
    n = nv(G)
    map = 0
    for i = 1:n
        N = neighborhood(G,i,1)
        D = Dnew[:,i]
        p = sortperm(D)
        P = Dict()
        for j = 1:n
            P[p[j]] = j
        end
        d = length(N)
        for j = 1:d
            R = P[N[j]]
            a = Set(N)
            b = Set(p[1:R])
            map += length(intersect(a,b))/(d*R)
        end
    end
    
    return map/n
end

function avg_distortion(Dnew,Dold)
    n,n = size(Dnew)
    d = 0
    for i = 1:n
        for j = 1:i-1
            d += abs(Dnew[i,j]-Dold[i,j])/Dold[i,j]
        end
    end
    
    return 2*d/(n*(n-1))
end

avg_distortion (generic function with 1 method)

In [70]:
using Random

@time G2,W2 = metric_to_structure(copy(Dtree));
@show(is_connected(G2),nv(G2),ne(G2));
D2 = LightGraphs.Parallel.floyd_warshall_shortest_paths(G2,W2[1:nv(G2),1:nv(G2)]).dists;
@show(avg_distortion(D2[1:n,1:n],A));
@show(MAP(D2[1:n,1:n],G));

(nv(G), ne(G)) = (1028, 1027)
  0.034168 seconds (192.85 k allocations: 18.558 MiB)
is_connected(G2) = true
nv(G2) = 1028
ne(G2) = 1027
avg_distortion(D2[1:n, 1:n], A) = 0.11215361546933861
MAP(D2[1:n, 1:n], G) = 0.8259833323476365


In [63]:
@time G2,W2 = metric_to_structure(copy(A));
@show(is_connected(G2),nv(G2),ne(G2));
D2 = LightGraphs.Parallel.floyd_warshall_shortest_paths(G2,W2).dists;
@show(avg_distortion(D2[1:n,1:n],A));
@show(MAP(D2[1:n,1:n],G));

(nv(G), ne(G)) = (668, 667)
  0.026153 seconds (118.27 k allocations: 15.332 MiB)
is_connected(G2) = true
nv(G2) = 668
ne(G2) = 667
avg_distortion(D2[1:n, 1:n], A) = 0.12343313624280457
MAP(D2[1:n, 1:n], G) = 0.916017400861897


In [71]:
MAP(Dtree,G)

0.9503551161876188

In [72]:
avg_distortion(Dtree,A)

0.0945460021268155

In [32]:
using JLD2, FileIO
@save "l1_tree_disease_TreeRepversion1.jld2" R